# Chargement et exploration des données

In [1]:
import pandas as pd

# Chargement des données
data = pd.read_csv('IMDB Dataset.csv')

# Aperçu des premières lignes
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Prétraitement des données

Le nettoyage en convertissant le texte en minuscules, en supprimant les balises HTML, les caractères spéciaux, les chiffres et les mots vides.

In [2]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Fonction de nettoyage du texte
def clean_text(text):
    # Mise en minuscule
    text = text.lower()
    # Suppression des balises HTML
    text = re.sub(r'<br />', '', text)
    # Suppression des caractères spéciaux et des chiffres
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Suppression des mots vides
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Application du nettoyage sur les critiques
data['cleaned_review'] = data['review'].apply(clean_text)

# Aperçu des premières lignes nettoyées
print(data['cleaned_review'].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    one reviewers mentioned watching oz episode yo...
1    wonderful little production filming technique ...
2    thought wonderful way spend time hot summer we...
3    basically theres family little boy jake thinks...
4    petter matteis love time money visually stunni...
Name: cleaned_review, dtype: object


# Tokenization et encodage des labels

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Séparation des données en train/test
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'], data['sentiment'], test_size=0.2, random_state=42)

# Conversion des labels 'positive' et 'negative' en 1 et 0
y_train = y_train.map({'positive': 1, 'negative': 0})
y_test = y_test.map({'positive': 1, 'negative': 0})

# Tokenization des textes
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# Conversion des textes en séquences d'entiers
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding pour avoir des séquences de même longueur
# Modifier la longueur des séquences
X_train_pad = pad_sequences(X_train_seq, maxlen=300)
X_test_pad = pad_sequences(X_test_seq, maxlen=300)

# Création du modèle de classification

Utilisation de  la couche **Embedding** pour transformer les mots en vecteurs, suivie d'une couche **LSTM** pour capturer les relations séquentielles. Une couche **Dropout** est ajoutée pour éviter le surapprentissage, et une couche **Dense** avec activation **sigmoid** produit la sortie binaire.

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Création du modèle LSTM
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compilation du modèle
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Résumé du modèle
model.summary()

C:\Users\aya\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)               ┃ Output Shape        ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ embedding (Embedding)      │ ?                   │ 0 (unbuilt) │
├────────────────────────────┼─────────────────────┼─────────────┤
│ lstm (LSTM)                │ ?                   │ 0 (unbuilt) │
├────────────────────────────┼─────────────────────┼─────────────┤
│ dropout (Dropout)          │ ?                   │ 0 (unbuilt) │
├────────────────────────────┼─────────────────────┼─────────────┤
│ dense (Dense)              │ ?                   │ 0 (unbuilt) │
└────────────────────────────┴─────────────────────┴─────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Entraînement du modèle

In [7]:
# Entraînement du modèle
entr = model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test))

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 424s 678ms/step - accuracy: 0.9473 - loss: 0.1436 - val_accuracy: 0.8793 - val_loss: 0.3011
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 434s 694ms/step - accuracy: 0.9593 - loss: 0.1142 - val_accuracy: 0.8703 - val_loss: 0.3465
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 432s 691ms/step - accuracy: 0.9679 - loss: 0.0925 - val_accuracy: 0.8676 - val_loss: 0.3894
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 424s 663ms/step - accuracy: 0.9787 - loss: 0.0672 - val_accuracy: 0.8738 - val_loss: 0.4398
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 415s 665ms/step - accuracy: 0.9824 - loss: 0.0555 - val_accuracy: 0.8682 - val_loss: 0.4755


# Évaluation et prédiction

In [8]:
# Évaluation sur les données de test
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Précision sur les données de test : {accuracy * 100:.2f}%")

# Prédiction sur de nouvelles critiques
new_reviews = ["This movie was fantastic!", "I didn't like this movie at all."]
new_reviews_cleaned = [clean_text(review) for review in new_reviews]
new_reviews_seq = tokenizer.texts_to_sequences(new_reviews_cleaned)
new_reviews_pad = pad_sequences(new_reviews_seq, maxlen=200)

predictions = model.predict(new_reviews_pad)
print(predictions)

313/313 ━━━━━━━━━━━━━━━━━━━━ 44s 142ms/step - accuracy: 0.8713 - loss: 0.4451
Précision sur les données de test : 86.82%
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[[0.62932754]
 [0.07341436]]


In [10]:
# Affichage des résultats
for i, review in enumerate(new_reviews):
    sentiment = "Positif" if predictions[i] >= 0.5 else "Négatif"
    print(f"Critique: {review}")
    print(f"Sentiment prédit: {sentiment} ({predictions[i][0] * 100:.2f}%)\n")


Critique: This movie was fantastic!
Sentiment prédit: Positif (62.93%)

Critique: I didn't like this movie at all.
Sentiment prédit: Négatif (7.34%)

